In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

In [19]:
df2 = pd.read_csv('Dart_codelist_last30_2.csv', usecols = ['종목명','종목코드','기업코드','수정일'], converters={'종목코드':str,'기업코드':str})
# 상장회사 목록

# usecols 뽑을 데이터 지정하는 코드, converters 종목코드 str으로 바꿔주는 코드

#종목코드 앞자리가 0이므로 스트링으로 지정해주지않으면 0이 사라짐
df2

,종목명,종목코드,기업코드,수정일
0,지놈앤컴퍼니,314130,01276026,20210216
1,웹젠,069080,00405320,20210215
2,우리넷,115440,00664048,20210217
3,마이크로디지탈,305090,01267967,20210217
4,나노,187790,00994994,20210216
5,손오공,066910,00306870,20210216
6,새로닉스,042600,00147994,20210217
7,피씨디렉트,051380,00297934,20210217
8,이큐셀,160600,00818472,20210125
9,케이엠제약,225430,01095704,20210126


In [20]:
# 종목코드의 기업명 불러낼 dict 만들기

corp_dict = {}

for i in range(len(df2['종목코드'])):
    corp_dict[df2.loc[i]['종목코드']] = str(df2.loc[i]['종목명'])

In [21]:
com_code = df2['종목코드'][:100]

In [22]:
# for i in 종목코드:
#     globals()['df{}'.format(★종목명 or ★종목코드)] = pd.DataFrame()

In [23]:
page_number_lst = []  # 18페이지 까지 돌리기
for i in range(10):
    page_number_lst.append(i+1)

In [24]:
# page_number_lst = np.arange(1,10) # 시작 페이지 ~ 끝 페이지 넘버 array

a = [1,2,3,4,5]

In [25]:
#############수정용



# page_number_lst = [1, 2, 3, 4, 5] 

list_date = []
list_endprice = []
corp_total = []
corp_name = []

for code in com_code:
    try:
        for _page in page_number_lst: # 1페이지부터 n페이지까지 크롤링

            df = pd.DataFrame()

            url_head = 'https://finance.naver.com/item/sise_day.nhn?code=361610&page={}'.format(_page)
            headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36'}

            web = requests.get(url_head, headers = headers).content #자료 요청
            soup = BeautifulSoup( web, 'html.parser') # html 파싱

            for tr in soup.find('table').find_all('tr'):
                try:
                    if len(tr.find_all("td")[0].get_text()) != 10: # 날짜가 비워진 셀은 패스 
                        pass
                    else:
                        if tr.find_all("td")[0].get_text() in list_date : # 동일한 날짜가 기존에 있었는지 체크 (중복 페이지)
                            break
                        else:
                            list_date.append(tr.find_all("td")[0].get_text())
                            list_endprice.append(tr.find_all('td')[1].get_text())
                            corp_total.append(code)
                            corp_name.append(corp_dict[code])

                        for i in range(len(quote_cp)):
                            quote_cp[i] = quote_cp[i].replace(",","")
                        quote_cp = list(map(int,quote_cp))
                except:
                    pass
        time.sleep(3)
    except:
        pass
